# Phase 11: Bear Profit Tuned Backtest

Test the tuned shorting strategy for improved bear market profits:
- **Lowered thresholds**: ATR >4% (from 5%) + RSI >65 (from 70)
- **VWAP filter**: Only short when price is above VWAP (mean reversion)
- **USDT yield**: Simulated ~5-8% APY while parked in defensive mode
- **Stronger short rewards**: 5x multiplier on profitable shorts
- **RSI auto-exit**: Close shorts when RSI < 40 (mean reversion)
- **15% max exposure**: Reduced from 20% for safety
- Goal: Flip Phase 10's -8.6% to flat/positive via tuned downside capture

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from data_fetcher import DataFetcher
from portfolio import Portfolio
from orchestrator import RLOrchestrator
from risk_manager import RiskManager
from strategies.dip_buy_lstm import generate_dip_signals, detect_dip_signal
from strategies.mean_reversion import calculate_vwap, is_above_vwap, get_vwap_deviation

print("Phase 11: Bear Profit Tuned Backtest")
print("="*50)
print("Key Changes from Phase 10:")
print("  - RSI threshold: 70 -> 65 (earlier overbought)")
print("  - ATR threshold: 5% -> 4% (earlier bear mode)")
print("  - VWAP filter: Only short above VWAP")
print("  - USDT yield: ~5-8% APY simulation")
print("  - RSI exit: Auto-close shorts when RSI < 40")

## 1. Fetch Historical Data (Nov-Dec Period)

In [ ]:
fetcher = DataFetcher()

# Fetch extended data to capture bear periods (Nov-Dec)
data = {}
for sym in ['XRP/USDT', 'BTC/USDT']:
    print(f"Fetching {sym}...")
    df = fetcher.fetch_ohlcv('kraken', sym, '1h', 2000)
    if not df.empty:
        data[sym] = df
        print(f"  {len(df)} candles from {df.index[0]} to {df.index[-1]}")

# Show price statistics
if 'XRP/USDT' in data:
    xrp = data['XRP/USDT']
    print(f"\nXRP range: ${xrp['close'].min():.4f} - ${xrp['close'].max():.4f}")
    print(f"XRP current: ${xrp['close'].iloc[-1]:.4f}")
    
if 'BTC/USDT' in data:
    btc = data['BTC/USDT']
    print(f"\nBTC range: ${btc['close'].min():.2f} - ${btc['close'].max():.2f}")
    print(f"BTC current: ${btc['close'].iloc[-1]:.2f}")

## 2. Phase 11: VWAP Mean Reversion Analysis

In [ ]:
# Phase 11: Analyze VWAP deviations for mean reversion shorts
if 'XRP/USDT' in data:
    df = data['XRP/USDT']
    
    vwap_analysis = []
    for i in range(50, len(df)):
        # Create data slice for VWAP calculation
        slice_data = {'XRP/USDT': df.iloc[:i+1]}
        
        deviation_pct, vwap = get_vwap_deviation(slice_data, 'XRP/USDT', period=20)
        current_price = df['close'].iloc[i]
        
        vwap_analysis.append({
            'timestamp': df.index[i],
            'price': current_price,
            'vwap': vwap,
            'deviation_pct': deviation_pct * 100,
            'above_vwap': deviation_pct > 0
        })
    
    vwap_df = pd.DataFrame(vwap_analysis)
    vwap_df.set_index('timestamp', inplace=True)
    
    # Plot VWAP analysis
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    axes[0].plot(vwap_df.index, vwap_df['price'], label='XRP Price', alpha=0.8)
    axes[0].plot(vwap_df.index, vwap_df['vwap'], label='VWAP (20)', color='orange', linewidth=2)
    axes[0].fill_between(vwap_df.index, vwap_df['price'], vwap_df['vwap'], 
                         where=vwap_df['above_vwap'], alpha=0.3, color='red', label='Above VWAP (Short Zone)')
    axes[0].fill_between(vwap_df.index, vwap_df['price'], vwap_df['vwap'], 
                         where=~vwap_df['above_vwap'], alpha=0.3, color='green', label='Below VWAP')
    axes[0].set_title('Phase 11: VWAP Mean Reversion Filter')
    axes[0].set_ylabel('Price ($)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(vwap_df.index, vwap_df['deviation_pct'], label='VWAP Deviation %', color='purple')
    axes[1].axhline(y=0, color='black', linestyle='-', linewidth=2)
    axes[1].axhline(y=2, color='red', linestyle='--', label='>2% = Strong Short Signal')
    axes[1].axhline(y=-2, color='green', linestyle='--', label='<-2% = Strong Long Signal')
    axes[1].fill_between(vwap_df.index, 0, vwap_df['deviation_pct'], 
                         where=vwap_df['deviation_pct'] > 0, alpha=0.3, color='red')
    axes[1].fill_between(vwap_df.index, 0, vwap_df['deviation_pct'], 
                         where=vwap_df['deviation_pct'] < 0, alpha=0.3, color='green')
    axes[1].set_title('VWAP Deviation % (Phase 11: Short only when positive)')
    axes[1].set_ylabel('Deviation %')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nVWAP Statistics:")
    print(f"  Time above VWAP: {vwap_df['above_vwap'].mean()*100:.1f}%")
    print(f"  Time below VWAP: {(~vwap_df['above_vwap']).mean()*100:.1f}%")
    print(f"  Avg deviation: {vwap_df['deviation_pct'].mean():.2f}%")
    print(f"  Max above VWAP: {vwap_df['deviation_pct'].max():.2f}%")
    print(f"  Max below VWAP: {vwap_df['deviation_pct'].min():.2f}%")

## 3. Phase 11: Tuned Bear Signal Detection (RSI>65, ATR>4%)

In [ ]:
risk = RiskManager(max_drawdown=0.20, max_leverage=10.0)

def calculate_rsi(close, period=14):
    """Calculate RSI."""
    if len(close) < period + 1:
        return 50.0
    deltas = np.diff(close[-period-1:])
    gains = np.where(deltas > 0, deltas, 0)
    losses = np.where(deltas < 0, -deltas, 0)
    avg_gain = np.mean(gains) if len(gains) > 0 else 0
    avg_loss = np.mean(losses) if len(losses) > 0 else 0.001
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

# Phase 11: Compare old vs new thresholds
if 'XRP/USDT' in data:
    df = data['XRP/USDT']
    
    bear_signals_old = []  # Phase 10: RSI>70, ATR>5%
    bear_signals_new = []  # Phase 11: RSI>65, ATR>4%
    bear_signals_vwap = []  # Phase 11 + VWAP filter
    
    for i in range(50, len(df) - 24):
        close = df['close'].iloc[:i+1].values
        high = df['high'].iloc[:i+1].values
        low = df['low'].iloc[:i+1].values
        
        rsi = calculate_rsi(close)
        atr_pct = risk.calculate_atr_pct(high, low, close)
        
        # VWAP check
        slice_data = {'XRP/USDT': df.iloc[:i+1]}
        above_vwap = is_above_vwap(slice_data, 'XRP/USDT')
        
        # Future outcome
        future_prices = df['close'].iloc[i:i+24].values
        current_price = close[-1]
        min_future = np.min(future_prices)
        drop_pct = (current_price - min_future) / current_price * 100
        
        # Phase 10 thresholds (old)
        if rsi > 70 and atr_pct > 0.05:
            bear_signals_old.append({
                'timestamp': df.index[i],
                'price': current_price,
                'rsi': rsi,
                'atr_pct': atr_pct * 100,
                'drop_pct': drop_pct,
                'success': drop_pct > 3
            })
        
        # Phase 11 thresholds (new)
        if rsi > 65 and atr_pct > 0.04:
            bear_signals_new.append({
                'timestamp': df.index[i],
                'price': current_price,
                'rsi': rsi,
                'atr_pct': atr_pct * 100,
                'drop_pct': drop_pct,
                'success': drop_pct > 3
            })
            
            # Phase 11 + VWAP filter
            if above_vwap:
                bear_signals_vwap.append({
                    'timestamp': df.index[i],
                    'price': current_price,
                    'rsi': rsi,
                    'atr_pct': atr_pct * 100,
                    'drop_pct': drop_pct,
                    'success': drop_pct > 3
                })
    
    print("\nBear Signal Comparison:")
    print("="*60)
    
    old_df = pd.DataFrame(bear_signals_old)
    new_df = pd.DataFrame(bear_signals_new)
    vwap_df_signals = pd.DataFrame(bear_signals_vwap)
    
    print(f"\nPhase 10 (RSI>70, ATR>5%):")
    if len(old_df) > 0:
        print(f"  Signals: {len(old_df)}")
        print(f"  Success rate: {old_df['success'].mean()*100:.1f}%")
        print(f"  Avg drop: {old_df['drop_pct'].mean():.2f}%")
    else:
        print(f"  Signals: 0")
    
    print(f"\nPhase 11 (RSI>65, ATR>4%):")
    if len(new_df) > 0:
        print(f"  Signals: {len(new_df)} (+{len(new_df) - len(old_df)} more opportunities)")
        print(f"  Success rate: {new_df['success'].mean()*100:.1f}%")
        print(f"  Avg drop: {new_df['drop_pct'].mean():.2f}%")
    else:
        print(f"  Signals: 0")
    
    print(f"\nPhase 11 + VWAP Filter:")
    if len(vwap_df_signals) > 0:
        print(f"  Signals: {len(vwap_df_signals)} (filtered by VWAP)")
        print(f"  Success rate: {vwap_df_signals['success'].mean()*100:.1f}%")
        print(f"  Avg drop: {vwap_df_signals['drop_pct'].mean():.2f}%")
    else:
        print(f"  Signals: 0")

## 4. Phase 11 Backtest - Tuned Bear Profit Strategy

In [ ]:
# Initialize portfolio and orchestrator
starting = {'USDT': 1000.0, 'XRP': 500.0, 'BTC': 0.0}
portfolio = Portfolio(starting.copy())

try:
    orchestrator = RLOrchestrator(portfolio, data)
    rl_enabled = orchestrator.enabled
    print(f"\nRL Orchestrator: {'Enabled' if rl_enabled else 'Disabled'}")
    if rl_enabled:
        print(f"  Targets: {orchestrator.get_target_allocation()}")
        print(f"\nPhase 11 Thresholds (tuned):")
        print(f"  Bear confidence threshold: {orchestrator.BEAR_CONFIDENCE_THRESHOLD}")
        print(f"  Bear vol threshold: {orchestrator.BEAR_VOL_THRESHOLD * 100}%")
        print(f"  RSI overbought: {orchestrator.RSI_OVERBOUGHT}")
        print(f"  RSI short exit: {orchestrator.RSI_SHORT_EXIT}")
except Exception as e:
    print(f"Could not initialize orchestrator: {e}")
    rl_enabled = False

In [ ]:
if rl_enabled and 'XRP/USDT' in data:
    df = data['XRP/USDT']
    btc_df = data.get('BTC/USDT')
    
    # Backtest loop
    results = []
    actions_log = []
    short_log = []
    usdt_yield_total = 0.0  # Phase 11: Track USDT yield
    
    step_size = 4  # Every 4 hours
    
    for i in range(100, len(df), step_size):
        timestamp = df.index[i]
        xrp_price = df['close'].iloc[i]
        btc_price = btc_df['close'].iloc[i] if btc_df is not None and i < len(btc_df) else 90000.0
        
        prices = {'XRP': xrp_price, 'BTC': btc_price, 'USDT': 1.0}
        
        # Update env step
        if orchestrator.env is not None:
            orchestrator.env.current_step = min(i, orchestrator.env.max_steps - 1)
        
        # Get RL decision
        result = orchestrator.decide_and_execute(prices)
        orchestrator.check_and_manage_positions(prices)
        orchestrator.update_env_step()
        
        # Phase 11: Simulate USDT yield (5-8% APY)
        usdt_balance = portfolio.balances.get('USDT', 0)
        if result.get('mode') == 'defensive' and usdt_balance > 100:
            yield_per_step = usdt_balance * 0.0001  # ~5-8% APY
            usdt_yield_total += yield_per_step
        
        # Log
        total_value = portfolio.get_total_usd(prices)
        results.append({
            'timestamp': timestamp,
            'xrp_price': xrp_price,
            'btc_price': btc_price,
            'portfolio_value': total_value,
            'usdt': portfolio.balances.get('USDT', 0),
            'xrp': portfolio.balances.get('XRP', 0),
            'btc': portfolio.balances.get('BTC', 0),
            'volatility': result.get('volatility', 0),
            'mode': result.get('mode', 'unknown'),
            'market_state': result.get('market_state', 'unknown'),
            'rsi_xrp': result.get('rsi', {}).get('XRP', 50),
            'rsi_btc': result.get('rsi', {}).get('BTC', 50),
            'usdt_yield': usdt_yield_total
        })
        
        if result.get('executed'):
            action_entry = {
                'timestamp': timestamp,
                'action': f"{result['asset']} {result['action_type']}",
                'leverage': result.get('leverage', 0),
                'mode': result.get('mode', 'unknown'),
                'trigger': result.get('offensive_trigger', result.get('short', 'standard')),
                'price': xrp_price,
                'confidence': result.get('confidence', 0)
            }
            actions_log.append(action_entry)
            
            if result.get('short'):
                short_log.append({
                    'timestamp': timestamp,
                    'asset': result['short'],
                    'entry_price': xrp_price,
                    'leverage': result.get('leverage', 0),
                    'collateral': result.get('collateral', 0)
                })
    
    results_df = pd.DataFrame(results)
    results_df.set_index('timestamp', inplace=True)
    
    print(f"\nBacktest complete: {len(results)} data points")
    print(f"Actions executed: {len(actions_log)}")
    print(f"Short positions opened: {len(short_log)}")
    print(f"Total USDT yield earned: ${usdt_yield_total:.2f}")

In [ ]:
if 'results_df' in dir() and len(results_df) > 0:
    # Plot results
    fig, axes = plt.subplots(5, 1, figsize=(14, 16))
    
    # Portfolio value
    axes[0].plot(results_df.index, results_df['portfolio_value'], label='Portfolio Value', linewidth=2)
    axes[0].set_title('Phase 11: Portfolio Value - Tuned Bear Profit Strategy')
    axes[0].set_ylabel('Value ($)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Holdings
    axes[1].plot(results_df.index, results_df['usdt'], label='USDT', color='green')
    axes[1].plot(results_df.index, results_df['xrp'] * results_df['xrp_price'], label='XRP (USD)', color='blue')
    axes[1].plot(results_df.index, results_df['btc'] * results_df['btc_price'], label='BTC (USD)', color='orange')
    axes[1].set_title('Holdings (USD Value)')
    axes[1].set_ylabel('Value ($)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Trading mode
    mode_colors = results_df['mode'].map({'offensive': 'green', 'defensive': 'orange', 'bear': 'red', 'unknown': 'gray'})
    mode_vals = results_df['mode'].map({'offensive': 1, 'defensive': 0, 'bear': -1, 'unknown': 0})
    axes[2].bar(results_df.index, mode_vals, color=mode_colors.values, alpha=0.6, width=0.02)
    axes[2].set_title('Trading Mode (1=Offensive/Long, 0=Defensive, -1=Bear/Short)')
    axes[2].set_ylabel('Mode')
    axes[2].set_ylim(-1.5, 1.5)
    
    # RSI with Phase 11 thresholds (65/40)
    axes[3].plot(results_df.index, results_df['rsi_xrp'], label='XRP RSI', color='blue')
    axes[3].plot(results_df.index, results_df['rsi_btc'], label='BTC RSI', color='orange')
    axes[3].axhline(y=30, color='green', linestyle='--', label='Oversold (Long)')
    axes[3].axhline(y=40, color='cyan', linestyle=':', label='Short Exit (RSI<40)')
    axes[3].axhline(y=65, color='red', linestyle='--', label='Overbought (Short) - Phase 11')
    axes[3].fill_between(results_df.index, 0, 30, alpha=0.2, color='green')
    axes[3].fill_between(results_df.index, 65, 100, alpha=0.2, color='red')
    axes[3].set_title('RSI Indicators (Phase 11: >65 = Short Zone, <40 = Exit)')
    axes[3].set_ylabel('RSI')
    axes[3].legend()
    axes[3].grid(True, alpha=0.3)
    
    # Volatility with Phase 11 threshold (4%)
    axes[4].plot(results_df.index, results_df['volatility'] * 100, label='ATR %', color='purple')
    axes[4].axhline(y=2, color='green', linestyle='--', label='Greed (<2%)')
    axes[4].axhline(y=4, color='red', linestyle='--', label='Bear (>4%) - Phase 11')
    axes[4].fill_between(results_df.index, 4, results_df['volatility'].max() * 100, alpha=0.2, color='red')
    axes[4].set_title('Volatility (ATR %) - Phase 11: >4% = Bear Mode Eligible')
    axes[4].set_ylabel('ATR %')
    axes[4].legend()
    axes[4].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary stats
    initial_value = results_df['portfolio_value'].iloc[0]
    final_value = results_df['portfolio_value'].iloc[-1]
    final_with_yield = final_value + usdt_yield_total  # Include USDT yield
    max_value = results_df['portfolio_value'].max()
    min_value = results_df['portfolio_value'].min()
    pnl = final_value - initial_value
    pnl_with_yield = final_with_yield - initial_value
    roi = (pnl / initial_value) * 100
    roi_with_yield = (pnl_with_yield / initial_value) * 100
    max_dd = (max_value - min_value) / max_value * 100
    
    print(f"\n" + "="*60)
    print(f"PHASE 11 TUNED BEAR PROFIT STRATEGY RESULTS")
    print(f"="*60)
    print(f"Initial Value:     ${initial_value:.2f}")
    print(f"Final Value:       ${final_value:.2f}")
    print(f"USDT Yield:        ${usdt_yield_total:.2f}")
    print(f"Final + Yield:     ${final_with_yield:.2f}")
    print(f"")
    print(f"Total P&L:         ${pnl:+.2f} ({roi:+.1f}%)")
    print(f"P&L + Yield:       ${pnl_with_yield:+.2f} ({roi_with_yield:+.1f}%)")
    print(f"Max Value:         ${max_value:.2f}")
    print(f"Min Value:         ${min_value:.2f}")
    print(f"Max Drawdown:      {max_dd:.1f}%")
    print(f"")
    print(f"Time in Offensive (Long): {(results_df['mode'] == 'offensive').mean()*100:.1f}%")
    print(f"Time in Bear (Short):     {(results_df['mode'] == 'bear').mean()*100:.1f}%")
    print(f"Time in Defensive:        {(results_df['mode'] == 'defensive').mean()*100:.1f}%")

## 5. Short Position Analysis

In [ ]:
if 'actions_log' in dir() and len(actions_log) > 0:
    actions_df = pd.DataFrame(actions_log)
    
    print("\nAction Summary:")
    print(actions_df['action'].value_counts())
    
    print("\nMode Distribution:")
    print(actions_df['mode'].value_counts())
    
    print("\nBear Mode Actions (Shorts):")
    bear_actions = actions_df[actions_df['mode'] == 'bear']
    if len(bear_actions) > 0:
        print(f"  Count: {len(bear_actions)}")
        print(bear_actions.to_string())
    else:
        print("  No bear mode actions triggered")
    
    print("\nOffensive Actions (Longs):")
    offensive = actions_df[actions_df['mode'] == 'offensive']
    if len(offensive) > 0:
        print(f"  Count: {len(offensive)}")
        print(offensive['trigger'].value_counts())
    
    print("\nLeveraged Trades:")
    lev_trades = actions_df[actions_df['leverage'] > 0]
    print(f"  Count: {len(lev_trades)}")
    if len(lev_trades) > 0:
        print(f"  Avg leverage: {lev_trades['leverage'].mean():.1f}x")
        print(f"  Max leverage: {lev_trades['leverage'].max():.0f}x")
else:
    print("\nNo actions executed")

## 6. Benchmark Comparison (Phase 10 vs Phase 11)

In [ ]:
if 'XRP/USDT' in data and 'results_df' in dir():
    df = data['XRP/USDT']
    
    # Buy and hold benchmarks
    initial_xrp = df['close'].iloc[100]
    final_xrp = df['close'].iloc[-1]
    xrp_return = (final_xrp - initial_xrp) / initial_xrp * 100
    
    if 'BTC/USDT' in data:
        btc_df = data['BTC/USDT']
        initial_btc = btc_df['close'].iloc[100]
        final_btc = btc_df['close'].iloc[-1]
        btc_return = (final_btc - initial_btc) / initial_btc * 100
    else:
        btc_return = 0
    
    # Our strategy return (with yield)
    strategy_return = roi_with_yield if 'roi_with_yield' in dir() else 0
    
    # Previous phase references
    phase9_return = -9.2   # From Phase 9 backtest
    phase10_return = -8.6  # From Phase 10 backtest
    
    print(f"\n" + "="*60)
    print(f"BENCHMARK COMPARISON")
    print(f"="*60)
    print(f"XRP Buy & Hold:     {xrp_return:+.1f}%")
    print(f"BTC Buy & Hold:     {btc_return:+.1f}%")
    print(f"")
    print(f"Phase 9 Strategy:   {phase9_return:+.1f}%")
    print(f"Phase 10 Strategy:  {phase10_return:+.1f}%")
    print(f"Phase 11 Strategy:  {strategy_return:+.1f}% (with USDT yield)")
    print(f"")
    print(f"Phase 11 vs Phase 10: {strategy_return - phase10_return:+.1f}%")
    print(f"Phase 11 vs Phase 9:  {strategy_return - phase9_return:+.1f}%")
    print(f"Phase 11 vs XRP:      {strategy_return - xrp_return:+.1f}%")
    print(f"Phase 11 vs BTC:      {strategy_return - btc_return:+.1f}%")
    
    # Performance assessment
    if strategy_return > phase10_return:
        improvement = strategy_return - phase10_return
        print(f"\n[SUCCESS] Phase 11 improved by {improvement:+.1f}% over Phase 10")
        if strategy_return >= 0:
            print(f"[GOAL MET] Flipped Phase 10's -8.6% to positive!")
        elif strategy_return > -5:
            print(f"[PROGRESS] Reduced losses significantly")
    else:
        print(f"\n[REVIEW] Bear strategy tuning needs adjustment")

## 7. Phase 11 Short P&L Simulation with VWAP Filter

In [ ]:
# Simulate potential short profits with Phase 11 tuned signals + VWAP filter
if 'vwap_df_signals' in dir() and len(vwap_df_signals) > 0:
    print("\nSimulated Short P&L (Phase 11 tuned signals + VWAP filter):")
    print("="*60)
    
    # Assume 5x leverage, 6% collateral (Phase 11 reduced from 8%)
    collateral = 1000 * 0.06  # $60 per trade
    leverage = 5
    
    total_pnl = 0
    wins = 0
    losses = 0
    
    for _, row in vwap_df_signals.iterrows():
        # P&L = collateral * leverage * drop_pct
        pnl = collateral * leverage * (row['drop_pct'] / 100)
        total_pnl += pnl
        if pnl > 0:
            wins += 1
        else:
            losses += 1
    
    print(f"  Total trades: {len(vwap_df_signals)}")
    print(f"  Wins: {wins}, Losses: {losses}")
    print(f"  Win rate: {wins/len(vwap_df_signals)*100:.1f}%")
    print(f"  Total P&L: ${total_pnl:+.2f}")
    print(f"  Avg P&L per trade: ${total_pnl/len(vwap_df_signals):+.2f}")
    
    # Impact on Phase 10 result
    phase10_pnl = -191.17  # From Phase 10 backtest
    combined_pnl = phase10_pnl + total_pnl + usdt_yield_total
    print(f"\n  Phase 10 P&L: ${phase10_pnl:+.2f}")
    print(f"  + Short profits: ${total_pnl:+.2f}")
    print(f"  + USDT yield: ${usdt_yield_total:+.2f}")
    print(f"  = Combined P&L: ${combined_pnl:+.2f}")
    
    if combined_pnl > 0:
        print(f"\n  [SUCCESS] Phase 11 tuning would have flipped to positive!")
    elif combined_pnl > phase10_pnl:
        print(f"\n  [PROGRESS] Phase 11 reduces losses by ${combined_pnl - phase10_pnl:.2f}")
else:
    print("\nNo VWAP-filtered bear signals found")

## 8. Next Steps

1. **Retrain PPO**: Run 2M timesteps with Phase 11 tuned rewards
2. **Live Paper**: Start `python src/live_paper.py --interval 15`
3. **Monitor**: Check logs/trades.csv for bear mode triggers with new thresholds
4. **Tune Further**: Adjust if still too passive (try RSI>60, ATR>3.5%)

In [ ]:
print("\n" + "="*60)
print("PHASE 11 BACKTEST COMPLETE")
print("="*60)
print("")
print("To retrain with Phase 11 tuned rewards:")
print("  python src/main.py --mode train-rl --timesteps 2000000 --device cuda")
print("")
print("To start live paper trading:")
print("  python src/live_paper.py --interval 15")
print("")
print("Key Phase 11 thresholds (tuned from Phase 10):")
print(f"  Bear mode: ATR > 4% (was 5%) + RSI > 65 (was 70)")
print(f"  VWAP filter: Only short when price > VWAP")
print(f"  RSI exit: Auto-close shorts when RSI < 40")
print(f"  Short leverage: 3-5x (unchanged)")
print(f"  Max short exposure: 15% (was 20%)")
print(f"  USDT yield: ~5-8% APY while parked")
print(f"  Short rewards: 5x multiplier on profits")